In [1]:
import pandas as pd
import os
import numpy as np
import pickle

In [ ]:
# read the txt readme files

demo=pd.DataFrame()

numbers=[2,3,4,5,6,7,8,9,10,11,13,14,15,16,17]

for i in numbers:
    filename = 'data\WESAD\S%i\S%i_readme.txt'%(i,i)

#filename = 'data\WESAD\S8\S8_readme.txt'

    with open(filename, 'r') as file:
        lines = file.readlines()

    subject_number = int(os.path.basename(filename).split('S')[1].split('_')[0])
    data = {'id': subject_number}
    current_header = None

    for line in lines:
        parts = line.strip().split(':')  # Split using ':' as delimiter
        if len(parts) == 1:  # If ':' is not found, split using '?' as delimiter
            parts = line.strip().split('?')
        if len(parts) > 1:
            header = parts[0].strip()  # Extract the column header
            value = ':'.join(parts[1:]).strip()  # Join remaining parts as value
            data[header] = value

    # Create a Pandas DataFrame
    user_df = pd.DataFrame([data])
    # before the next user, add this user_df to the total dataframe
    demo = pd.concat([demo, user_df])

demo = demo.drop(columns=['Stress condition / TSST interview part'])
demo

In [ ]:
# read the txt respiban files

respiban=pd.DataFrame()

for i in numbers:
    filename = 'data\WESAD\S%i\S%i_respiban.txt'%(i,i)
    user_df=pd.DataFrame()
    user_df = pd.read_csv(filename, sep='\t', header=None, skiprows=3)
    user_df['id'] = i

    # before the next user, add this user_df to the total dataframe
    respiban = pd.concat([respiban, user_df])

respiban = respiban.drop(columns=[0,1,10])
respiban.columns = ['ECG', 'EDA', 'EMG', 'TEMP', 'XYZ1', 'XYZ2', 'XYZ3', 'RESPIRATION', 'id']
respiban

In [ ]:
chan_bit=2**16
vcc=3

respiban['ECG (mV)'] = ((respiban['ECG'] / chan_bit - 0.5) * vcc)
respiban['EDA (μS)'] = (((respiban['EDA']/chan_bit)*vcc)/0.12)
respiban['EMG (mV)'] = ((respiban['EMG']/chan_bit-0.5)*vcc)
vout = (respiban['TEMP']*vcc)/(chan_bit-1.)
rntc = ((10**4)*vout)/(vcc-vout)
respiban['TEMP (^oC)'] = -273.15 + 1./(1.12764514*(10**(-3)) + 2.34282709*(10**(-4))*np.log(rntc)+8.77303013*(10**(-8))*(np.log(rntc)**3))
respiban

In [ ]:
# read the PANAS self questionnaires csv files

# PANAS scores can range from 10 to 50, with higher scores representing higher levels of positive or negative affect, respectively.

def calculate_panas_scores_from_dataframe(df):

    positive_headers = ["Interested", "Inspired", "Guilty", "Excited", "Irritable", "Alert", "Attentive", "Jittery", "Afraid", "Stressed"]
    negative_headers = ["Active", "Distressed", "Annoyed", "Strong", "Scared", "Hostile", "Proud", "Enthusiastic", "Ashamed", "Sad", "(Angry)", "(Irritated)"]

    df[positive_headers + negative_headers] = df[positive_headers + negative_headers].apply(pd.to_numeric, errors='coerce').fillna(0)

    pa_score = df[positive_headers].astype(int).sum(axis=1)
    na_score = df[negative_headers].astype(int).sum(axis=1)

    df['PA_Score'] = pa_score
    df['NA_Score'] = na_score

    return df

numbers=[2,3,4,5,6,7,8,9,10,11,13,14,15,16,17]

panas=pd.DataFrame()

for user in numbers:
    filename = 'data\WESAD\S%i\S%i_quest.csv'%(i,i)
    #user_df=pd.DataFrame()

    lines_to_read = 5
    data = []

    header_mapping = {
    1: "Active",
    2: "Distressed",
    3: "Interested",
    4: "Inspired",
    5: "Annoyed",
    6: "Strong",
    7: "Guilty",
    8: "Scared",
    9: "Hostile",
    10: "Excited",
    11: "Proud",
    12: "Irritable",
    13: "Enthusiastic",
    14: "Ashamed",
    15: "Alert",
    16: "Nervous",
    17: "Determined",
    18: "Attentive",
    19: "Jittery",
    20: "Afraid",
    21: "Stressed",
    22: "Frustrated",
    23: "Happy",
    24: "Sad",
    25: "(Angry)",
    26: "(Irritated)"
}

    with open(filename, 'r') as csvfile:
        lines_read = 0
        for line in csvfile:
            if lines_read >= lines_to_read:
                break
            if line.strip().startswith('# PANAS'):
                values = line.strip().split(';')[1:]
                data.append(values)
                lines_read += 1

    user_df = pd.DataFrame(data, columns=[header_mapping[i] for i in range(1, len(data[0]) + 1)])
    user_df['id'] = user

    # before the next user, add this user_df to the total dataframe
    panas = pd.concat([panas, user_df])

panas = calculate_panas_scores_from_dataframe(panas)
panas = panas.drop(panas.iloc[:, 0:26],axis = 1)

panas

In [ ]:
# read the STAI self questionnaires csv files

# to calculate the total stress score simply sum per row

numbers=[2,3,4,5,6,7,8,9,10,11,13,14,15,16,17]

stai=pd.DataFrame()

for user in numbers:
    filename = 'data\WESAD\S%i\S%i_quest.csv'%(user,user)

    lines_to_read = 5
    data = []

    with open(filename, 'r') as csvfile:
        lines_read = 0
        for line in csvfile:
            if lines_read >= lines_to_read:
                break
            if line.strip().startswith('# STAI'):
                values = line.strip().split(';')[1:]
                data.append(values)
                lines_read += 1

    user_df = pd.DataFrame(data)
    user_df = user_df.iloc[:, 0:6]
    user_df['id'] = user

    # before the next user, add this user_df to the total dataframe
    stai = pd.concat([stai, user_df])

for column in stai.columns:
    stai[column] = pd.to_numeric(stai[column], errors='coerce')


stai['stai_stress'] = stai.iloc[:, 0:6].sum(axis=1)
stai = stai.iloc[:, 6:]

mean_stai = stai['stai_stress'].mean()
std_stai = stai['stai_stress'].std()

def get_stai_category(score):
    if score < mean_stai-0.5*std_stai:
        return "Below average"
    if score > mean_stai+0.5*std_stai:
        return "Above average"
    return "Average"

stai['stai_stress_category'] = stai['stai_stress'].apply(lambda score: get_stai_category(score))

stai = stai.drop(columns=['stai_stress'])

stai

In [2]:
# read the sync data from the two devices

data = pd.read_pickle('data\WESAD\S2\S2.pkl')

def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Flatten the nested dictionary
#flattened_dict = flatten_dict(data)

# Convert the flattened dictionary into a DataFrame
#data = pd.DataFrame([flattened_dict])

# Explode arrays into rows
#for col in data.columns:
#    data = data.explode(col)

data

{'signal': {'chest': {'ACC': array([[ 0.95539999, -0.222     , -0.55799997],
          [ 0.92579997, -0.2216    , -0.55379999],
          [ 0.90820003, -0.21960002, -0.53920001],
          ...,
          [ 0.87179995, -0.12379998, -0.30419999],
          [ 0.87300003, -0.12339997, -0.30260003],
          [ 0.87020004, -0.12199998, -0.30220002]]),
   'ECG': array([[ 0.02142334],
          [ 0.02032471],
          [ 0.01652527],
          ...,
          [-0.00544739],
          [ 0.00013733],
          [ 0.0040741 ]]),
   'EMG': array([[-0.00444031],
          [ 0.00434875],
          [ 0.00517273],
          ...,
          [-0.01716614],
          [-0.02897644],
          [-0.02357483]]),
   'EDA': array([[5.25054932],
          [5.26733398],
          [5.24330139],
          ...,
          [0.36048889],
          [0.36582947],
          [0.365448  ]]),
   'Temp': array([[30.120758],
          [30.129517],
          [30.138214],
          ...,
          [31.459229],
          [31.484283

In [3]:
c_ax=data['signal']['chest']['ACC'][0:,0]#[baseline_to_plot]
c_ay=data['signal']['chest']['ACC'][0:,1]#[baseline_to_plot]#[0:100] # hundred values
c_az=data['signal']['chest']['ACC'][0:,2]#[baseline_to_plot]
c_ecg=data['signal']['chest']['ECG'][:,0]#[baseline_to_plot]
c_emg=data['signal']['chest']['EMG'][:,0]#[baseline_to_plot]
c_eda=data['signal']['chest']['EDA'][:,0]#[baseline_to_plot]
c_temp=data['signal']['chest']['Temp'][:,0]#[baseline_to_plot]
c_resp=data['signal']['chest']['Resp'][:,0]#[baseline_to_plot]
w_ax=data['signal']['wrist']['ACC'][0:,0]#[stress_to_plot]
w_ay=data['signal']['wrist']['ACC'][0:,1]#[stress_to_plot]
w_az=data['signal']['wrist']['ACC'][0:,2]#[stress_to_plot]
w_bvp=data['signal']['wrist']['BVP'][:,0]#[stress_to_plot]
w_eda=data['signal']['wrist']['EDA'][:,0]#[stress_to_plot]
w_temp=data['signal']['wrist']['TEMP'][:,0]#[stress_to_plot]
w_label=data['label']

print("Done")

Done


In [6]:
respi = pd.DataFrame(data=np.column_stack((c_ax, c_ay, c_az, c_ecg, c_emg, c_eda, c_temp, c_resp)),columns=['respiban_ACC_X','respiban_ACC_Y', 'respiban_ACC_Z','respiban_ECG', 'respiban_EMG', 'respiban_EDA', 'respiban_temp', 'respiban_resp'])
respi

,respiban_ACC_X,respiban_ACC_Y,respiban_ACC_Z,respiban_ECG,respiban_EMG,respiban_EDA,respiban_temp,respiban_resp
0,0.9554,-0.2220,-0.5580,0.021423,-0.004440,5.250549,30.120758,-1.148987
1,0.9258,-0.2216,-0.5538,0.020325,0.004349,5.267334,30.129517,-1.124573
2,0.9082,-0.2196,-0.5392,0.016525,0.005173,5.243301,30.138214,-1.152039
3,0.8974,-0.2102,-0.5122,0.016708,0.007187,5.249405,30.129517,-1.158142
4,0.8882,-0.2036,-0.4824,0.011673,-0.015152,5.286407,30.130951,-1.161194
...,...,...,...,...,...,...,...,...
4255295,0.8750,-0.1234,-0.2974,-0.013138,0.020370,0.400162,31.457733,-1.063538
4255296,0.8750,-0.1262,-0.2988,-0.010345,0.019592,0.355911,31.476898,-1.106262
4255297,0.8718,-0.1238,-0.3042,-0.005447,-0.017166,0.360489,31.459229,-1.103210
4255298,0.8730,-0.1234,-0.3026,0.000137,-0.028976,0.365829,31.484283,-1.086426


In [ ]:
class read_data_of_one_subject:
    """Read data from WESAD dataset"""
    def __init__(self, path, subject):
        self.keys = ['label', 'subject', 'signal']
        self.signal_keys = ['wrist', 'chest']
        self.chest_sensor_keys = ['ACC', 'ECG', 'EDA', 'EMG', 'Resp', 'Temp']
        self.wrist_sensor_keys = ['ACC', 'BVP', 'EDA', 'TEMP']
        os.chdir(path)
        os.chdir(subject)
        with open(subject + '.pkl', 'rb') as file:
            data = pickle.load(file, encoding='latin1')
        self.data = data

    def get_labels(self):
        return self.data[self.keys[0]]

    def get_wrist_data(self):
        """"""
        #label = self.data[self.keys[0]]
        assert subject == self.data[self.keys[1]]
        signal = self.data[self.keys[2]]
        wrist_data = signal[self.signal_keys[0]]
        #wrist_ACC = wrist_data[self.wrist_sensor_keys[0]]
        #wrist_ECG = wrist_data[self.wrist_sensor_keys[1]]
        return wrist_data

    def get_chest_data(self):
        """"""
        signal = self.data[self.keys[2]]
        chest_data = signal[self.signal_keys[1]]
        return chest_data

In [ ]:
data_set_path = "data/WESAD"
subject = 'S3'

In [ ]:
obj_data = {}
obj_data[subject] = read_data_of_one_subject(data_set_path, subject)

In [ ]:
chest_data_dict = obj_data[subject].get_chest_data()
chest_dict_length = {key: len(value) for key, value in chest_data_dict.items()}
print(chest_dict_length)

In [ ]:
# Get labels
labels = obj_data[subject].get_labels()
baseline = np.asarray([idx for idx,val in enumerate(labels) if val == 1])
#print(baseline)

print("Baseline:", chest_data_dict['ECG'][baseline].shape)